In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-78lz8zkl
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-78lz8zkl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=2e45bb065ef19cec21e8f151f4e26665dfa361f612cc9f7b2ca8734b9ccb6a5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-w63ztcqx/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!nvidia-smi

Fri Apr 10 05:03:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
!ls

sample_data  src


In [11]:
%%cuda --name helloCUDA.cu
#include <iostream>
#include <random>
#include <algorithm>

#define LOOP(x) for(int t##x = 0; t##x < x; t##x++)

using namespace std;

void gpu_error(cudaError_t const &code) 
{
    if(code != cudaSuccess)
    {
        cerr << "GPUError: Code " << code << " : " << cudaGetErrorString(code) << endl;
        exit( EXIT_FAILURE );
    }
}

__global__ void tile(float *devin, float *devout, float *devsum, int h, int w)
{
    float thrtile[4][4];
    
    int /*bs,*/ p, q, ch;
    // bs = gridDim.x;
    p = gridDim.y;
    q = gridDim.z;
    ch = blockDim.x;
    
    int tbs, tp, tq, tch;
    tbs = blockIdx.x;
    tp = blockIdx.y;
    tq = blockIdx.z;
    tch = threadIdx.x;

    // copy the tiles to thrtile

    int offset1 = (tbs*ch + tch)*h*w;

    // float *t = thrtile;
 
    for(int th = 2*tp, i = 0; i < 4; th++, i++)
    {
        for(int tw = 2*tq, j = 0; j < 4; tw++, j++)
        {
            thrtile[i][j] = devin[offset1 + th*w + tw];
        }
    }

    // copy thrtile to devout for testing

    int offset2 = (((tbs*p + tp)*q + tq)*ch + tch)*16;

    for(int i = 0; i < 4; i++)
    {
        for(int j = 0; j < 4; j++)
        {
            devout[offset2 + i*4 + j] = thrtile[i][j];
        }
    }

    // sum along the channels, using log n summing

    // int k = ch, j = tch;

    int offset3 = ((tbs*p + tp)*q + tq)*ch*16;

    for(int s = 1; s < ch; s *= 2)
    {
        if(tch % (2*s) == 0 && tch+s < ch)
        {
            for(int i = 0; i < 4; i++)
            {
                for(int j = 0; j < 4; j++)
                {
                    devout[offset3 + tch*16 + i*4 + j] += devout[offset3 + (tch+s)*16 + i*4 + j];
                }
            }
        }
        __syncthreads();
    }

    if(tch/*%ch*/ == 0) // can do with tch == 0
    {
        int offset = ((tbs*p + tp)*q + tq)*16;
        for(int i = 0; i < 4; i++)
        {
            for(int j = 0; j < 4; j++)
            {
                devsum[offset + i*4 + j] = devout[offset3 + /*tch*16*/ +i*4 + j];
            }
        }
    }

}

void tilehost(int och, int ch, int bs, int h, int w, float *&in, int &p, int &q, int &outsize, float *&out, int &sumsize, float *&sum)
{
    // int p, q;
    p = max((h-2)/2, 0);
    q = max((w-2)/2, 0);
    
    float *devin, *devout, *devsum;
    devin = devout = devsum = nullptr;
    int insize = bs * ch * h * w * sizeof(float);
    outsize = bs * p * q * ch * 4 * 4 * sizeof(float);
    sumsize = bs * p * q * 4 * 4 * sizeof(float);

    gpu_error(cudaMalloc((void **) & devin, insize));
    gpu_error(cudaMalloc((void **) & devout, outsize));
    gpu_error(cudaMalloc((void **) & devsum, sumsize));
    
    gpu_error(cudaMemcpy(devin, in, insize, cudaMemcpyHostToDevice));

    // call the kernel function for tiling
    
    dim3 grid(bs, p, q);  // 3-D
    dim3 block(ch, 1, 1); // 1-D

    tile<<<grid, block>>>(devin, devout, devsum, h, w);

    // copy from device to host to out.

    delete in;
    out = new float[outsize/sizeof(float)];
    sum = new float[sumsize/sizeof(float)];

    gpu_error(cudaMemcpy(out, devout, outsize, cudaMemcpyDeviceToHost));
    gpu_error(cudaMemcpy(sum, devsum, sumsize, cudaMemcpyDeviceToHost));

    gpu_error(cudaFree(devin));
    gpu_error(cudaFree(devout));
    gpu_error(cudaFree(devsum));
    
}

void padding(float *&in, int bs, int ch, int &h, int &w, int pad)
{
    // Here, after adding pad we also round up h, w to become a multiple of tile.
    // This is done such that the actual matrix is present at top left of this matrix.

    int newh, neww;
    newh = h + 2*pad;
    neww = w + 2*pad;
    if(newh%2)
        newh++;
    if(neww%2)
        neww++;
    if(newh < 4)
        newh = 4;
    if(neww < 4)
        neww = 4;

    int slices = bs*ch;
    int newhw = newh*neww;
    float *newin = new float[slices*newhw];
    float *tin = in, *tnewin = newin;
    LOOP(slices)
    {
        LOOP(newh)
        {
            LOOP(neww)
            {
                if(tnewh >= pad && tnewh-pad < h && tneww >= pad && tneww-pad < w)
                {
                    *(tnewin++) = *(tin++);
                }
                else
                {
                    *(tnewin++) = 0;
                }
            }
        }
    }

    delete in;
    in = newin;

    h = newh;
    w = neww;

}

int main(void) 
{
    auto engine = default_random_engine(time(nullptr));
    auto rng = uniform_real_distribution<float>();

    int bs, ch, h, w, p, q, oldh, oldw;
    
    bs = 1;
    ch = 2;
    oldh = h = 1;
    oldw = w = 1;
    
    int insize = bs * ch * h * w * sizeof(float);
    int outsize, sumsize;
 
    float *in = new float[insize/sizeof(float)];
    float *t = in;
    float *out, *sum;
 
    LOOP(bs)
    {
        LOOP(ch)
        {
            LOOP(h)
            {
                LOOP(w)
                {
                    *(t++) = rng(engine);
                }
            }
        }
    }
 
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<in[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }

    cout<<"\nPadding\n";

    int pad = 1;

    padding(in, bs, ch, h, w, pad);

    cout<<"\nPadding done\n";
    
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<in[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }

    cout<<"\nTiling and Summing\n";

    tilehost(1, ch, bs, h, w, in, p, q, outsize, out, sumsize, sum);
    
    cout<<"\nTiling finished\n\n";

    /*
    
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                LOOP(ch)
                {
                    cout<<"{ ";
                    for(int i = 0; i < 4; i++)
                    {
                        for(int j = 0; j < 4; j++)
                        {
                            cout<<out[((((tbs*p+tp)*q+tq)*ch+tch)*4+i)*4+j]<<",";
                        }
                        cout<<";\n";
                    }
                    cout<<"}\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
 
    */

    cout<<"\nSumming finished\n\n";

    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                for(int i = 0; i < 4; i++)
                {
                    for(int j = 0; j < 4; j++)
                    {
                        cout<<sum[(((tbs*p+tp)*q+tq)*4+i)*4+j]<<",";
                    }
                    cout<<";\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }

    return 0;
}

'File written in /content/src/helloCUDA.cu'

In [12]:
!nvcc -std=c++11 /content/src/helloCUDA.cu -o /content/src/helloCUDA

/content/src/helloCUDA.cu(187): warning: variable "oldh" was set but never used

/content/src/helloCUDA.cu(187): warning: variable "oldw" was set but never used



In [13]:
!/content/src/helloCUDA #< input.txt

{ { { 0.497757 }
}
{ { 0.794631 }
}
}

Padding

Padding done
{ { { 0 0 0 0 }
{ 0 0.497757 0 0 }
{ 0 0 0 0 }
{ 0 0 0 0 }
}
{ { 0 0 0 0 }
{ 0 0.794631 0 0 }
{ 0 0 0 0 }
{ 0 0 0 0 }
}
}

Tiling and Summing

Tiling finished


Summing finished

{ { { 0,0,0,0,;
0,1.29239,0,0,;
0,0,0,0,;
0,0,0,0,;
}
}
}
